<a href="https://colab.research.google.com/github/Natasha617/DZ_NLP/blob/DZ_2/DZ_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Домашняя работа №2 Шафорост Н.В.**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp "/content/drive/MyDrive/магистратура/NLP/spam_or_not_spam.csv" /content/data.csv

In [87]:
import pandas as pd
import numpy as np
import re
import spacy
import nltk
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score

In [13]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
data = pd.read_csv("data.csv", encoding='utf-8')
data.head(2)

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0


In [15]:
data['email'][1]

'martin a posted tassos papadopoulos the greek sculptor behind the plan judged that the limestone of mount kerdylio NUMBER miles east of salonika and not far from the mount athos monastic community was ideal for the patriotic sculpture as well as alexander s granite features NUMBER ft high and NUMBER ft wide a museum a restored amphitheatre and car park for admiring crowds are planned so is this mountain limestone or granite if it s limestone it ll weather pretty fast yahoo groups sponsor NUMBER dvds free s p join now URL to unsubscribe from this group send an email to forteana unsubscribe URL your use of yahoo groups is subject to URL '

In [16]:
new_df=[]
for d in data['email']:
  if type(d) == str:
    new_df.append(d.split(" "))
  else:
    new_df.append([])

In [17]:
#все числа и URL-адреса были преобразованы в строки как NUMBERи URL соответственно
no_trash = [] # здесь будут данные без "URL" И "NUMBER"
for li in new_df:
    no = []
    for word in li:
      word = re.sub('[^a-zA-Zа-яА-Я]+', ' ', word)
      if word != 'URL' and word != "NUMBER":
        no.append(word)
    no_trash.append(no)

unique_stops = set(stopwords.words('english'))

# создаём пустой список без стоп-слов
no_stops = []
for li in no_trash:
    no_stop = []
    for word in li:
      word = word.lower()
      if word not in unique_stops and word.isalpha():
        no_stop.append(word)
    no_stops.append(no_stop)
print(no_stops[:2])

[['date', 'wed', 'aug', 'chris', 'garrigues', 'cwg', 'dated', 'numberfanumberd', 'deepeddy', 'com', 'message', 'id', 'tmda', 'deepeddy', 'vircio', 'com', 'reproduce', 'error', 'repeatable', 'like', 'every', 'time', 'without', 'fail', 'debug', 'log', 'pick', 'happening', 'exec', 'pick', 'inbox', 'list', 'lbrace', 'lbrace', 'subject', 'ftp', 'rbrace', 'rbrace', 'sequence', 'mercury', 'exec', 'pick', 'inbox', 'list', 'lbrace', 'lbrace', 'subject', 'ftp', 'rbrace', 'rbrace', 'sequence', 'mercury', 'hit', 'marking', 'hits', 'tkerror', 'syntax', 'error', 'expression', 'int', 'note', 'run', 'pick', 'command', 'hand', 'delta', 'pick', 'inbox', 'list', 'lbrace', 'lbrace', 'subject', 'ftp', 'rbrace', 'rbrace', 'sequence', 'mercury', 'hit', 'hit', 'comes', 'obviously', 'version', 'nmh', 'using', 'delta', 'pick', 'version', 'pick', 'nmh', 'compiled', 'sun', 'mar', 'ict', 'relevant', 'part', 'delta', 'mhparam', 'pick', 'seq', 'sel', 'list', 'since', 'pick', 'command', 'works', 'sequence', 'actually

In [30]:
lemmatizer = WordNetLemmatizer()

# и пустой список для слов после лемматизации
lemmatized = []
for li in no_stops:
    lemma = []
    for word in li:
      word = lemmatizer.lemmatize(word)
      lemma.append(word)
    l = ' '.join(lemma)
    lemmatized.append(l)
print(lemmatized[:2])

['date wed aug chris garrigues cwg dated numberfanumberd deepeddy com message id tmda deepeddy vircio com reproduce error repeatable like every time without fail debug log pick happening exec pick inbox list lbrace lbrace subject ftp rbrace rbrace sequence mercury exec pick inbox list lbrace lbrace subject ftp rbrace rbrace sequence mercury hit marking hit tkerror syntax error expression int note run pick command hand delta pick inbox list lbrace lbrace subject ftp rbrace rbrace sequence mercury hit hit come obviously version nmh using delta pick version pick nmh compiled sun mar ict relevant part delta mhparam pick seq sel list since pick command work sequence actually one explicit command line search popup one come get created kre p still using version code form day ago able reach cv repository today local routing issue think exmh worker mailing list exmh worker', 'martin posted tasso papadopoulos greek sculptor behind plan judged limestone mount kerdylio mile east salonika far mount

In [31]:
data['email'] = lemmatized

In [32]:
data

,email,label
0,date wed aug chris garrigues cwg dated numberf...,0
1,martin posted tasso papadopoulos greek sculpto...,0
2,man threatens explosion moscow thursday august...,0
3,klez virus die already prolific virus ever kle...,0
4,adding cream spaghetti carbonara effect pasta ...,0
...,...,...
2995,abc good morning america rank christmas toy se...,1
2996,hyperlink hyperlink hyperlink let mortgage len...,1
2997,thank shopping u gift occasion free gift purch...,1
2998,famous ebay marketing e course learn sell comp...,1



# CountVectorizer


In [50]:
vectorizer = CountVectorizer(analyzer = "word",
                             lowercase = True,
                             tokenizer = None,
                             preprocessor = None,
                             max_features = 5000)

bow_cv = vectorizer.fit_transform(data['email'])
# на выходе получается матрица csr
print(type(bow_cv))
print(bow_cv.toarray())
bow_cv.shape

<class 'scipy.sparse._csr.csr_matrix'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


(3000, 5000)

In [51]:
# здесь числа это не частотность, а просто порядковый номер (индекс)
vocab = vectorizer.vocabulary_
print(vocab)

{'date': 1082, 'wed': 4852, 'aug': 330, 'chris': 719, 'garrigues': 1858, 'cwg': 1060, 'dated': 1083, 'deepeddy': 1118, 'com': 803, 'message': 2762, 'id': 2142, 'tmda': 4531, 'vircio': 4780, 'reproduce': 3737, 'error': 1494, 'like': 2549, 'every': 1528, 'time': 4521, 'without': 4914, 'fail': 1619, 'debug': 1104, 'log': 2589, 'pick': 3290, 'happening': 1999, 'exec': 1553, 'inbox': 2198, 'list': 2561, 'subject': 4312, 'ftp': 1833, 'sequence': 3989, 'hit': 2071, 'syntax': 4384, 'expression': 1597, 'int': 2274, 'note': 2994, 'run': 3859, 'command': 813, 'hand': 1989, 'delta': 1139, 'come': 809, 'obviously': 3067, 'version': 4757, 'nmh': 2977, 'using': 4720, 'compiled': 847, 'sun': 4345, 'relevant': 3696, 'part': 3201, 'seq': 3988, 'sel': 3963, 'since': 4080, 'work': 4927, 'actually': 48, 'one': 3101, 'line': 2555, 'search': 3935, 'get': 1884, 'created': 1017, 'kre': 2441, 'still': 4266, 'code': 781, 'form': 1775, 'day': 1089, 'ago': 112, 'able': 5, 'reach': 3607, 'cv': 1057, 'repository': 3

In [52]:

tokens = vectorizer.get_feature_names_out()
# вначале создадим индекс предложений (документов)
index_list = []

# в цикле пройдемся по элементам матрицы, обозначим их через '_'
# функция enumerate задаст каждому элементу индекс, начиная с 0
for i, _ in enumerate(bow_cv):

    # прибавим наш индекс к слову Sentence
    index_list.append(f'Sentence_{i}')

# теперь можно использовать pd.DataFrame
bow_cv_df = pd.DataFrame(data = bow_cv.toarray(),
                         index = index_list,
                         columns = tokens)
bow_cv_df

,aa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,abandoned,abidjan,ability,able,abroad,absence,absolute,absolutely,...,zealand,zealot,zero,ziggy,zimbabwe,zip,zone,zope,zowie,zzzz
Sentence_0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sentence_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sentence_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sentence_3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sentence_4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sentence_2995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sentence_2996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sentence_2997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sentence_2998,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# TfidfVectorizer

In [81]:
tfIdfVectorizer = TfidfVectorizer(use_idf = True, stop_words= 'english')

# рассчитываем TF-IDF слов
tfIdf = tfIdfVectorizer.fit_transform(data['email'])

In [82]:
# можно посмотреть IDF слов
tfIdfVectorizer.idf_

array([6.60880557, 8.31355366, 8.31355366, ..., 5.96217841, 8.31355366,
       7.39726293])

In [84]:
#смотрим количество документов и токенов
tfIdf.shape

(3000, 29381)

In [86]:
mean_weights = np.asarray(tfIdf.mean(axis = 0)).ravel().tolist()
print(mean_weights)
# создаём датафрейм из словаря
mean_weights_df = pd.DataFrame({'term': tfIdfVectorizer.get_feature_names_out(), 'mean_weights': mean_weights})

# сортируем по убыванию 10 слов с максимальным средним TF-IDF
mean_weights_df.sort_values(by = 'mean_weights', ascending = False).reset_index(drop = True).head(10)


[0.0007714347761464064, 0.00010677408793741469, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 7.626720566958193e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.8133602834790965e-06, 3.813360283479096

,term,mean_weights
0,url,0.026371
1,list,0.026139
2,date,0.025924
3,numbertnumber,0.019923
4,rpm,0.019163
5,hyperlink,0.019063
6,email,0.016479
7,spamassassin,0.016223
8,message,0.016073
9,use,0.014704


**Разбиение на обучающую и тестовую выборки**

In [63]:
X = bow_cv_df.drop(columns = ['label'])
y = data['label']

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2023, test_size= 0.3)

# Дерево решений

In [65]:
clf = tree.DecisionTreeClassifier()
parametrs = { 'criterion': ['gini', 'entropy','log_loss'],
              'splitter': ['best', 'random'],
              'max_depth': [10,11],
              'min_samples_split': [2,3],
              'random_state': [40,50]
            }
grid_tree = GridSearchCV(clf, parametrs, cv=10,scoring="f1_weighted")
grid_tree.fit(X, y)
print(f'Лучшие параметры для дерева решений:{grid_tree.best_params_}')
print(f'Лучший результат предсказания:{grid_tree.best_score_}')

Лучшие параметры для дерева решений:{'criterion': 'entropy', 'max_depth': 11, 'min_samples_split': 2, 'random_state': 40, 'splitter': 'best'}
Лучший результат предсказания:0.9504493005772456


In [ ]:
clf = tree.DecisionTreeClassifier(**grid_tree.best_params_) #создание модели с лучшими параметрами
clf.fit(X_train,y_train)

In [69]:
cross_val_score(clf, X, y)

array([0.93333333, 0.92333333, 0.94166667, 0.95666667, 0.95833333])

# Логистическая регрессия

In [ ]:
clf1 = LogisticRegression() # то же самаое для логистической регрессии
parametrs = { 'penalty': ['l1', 'l2', 'elasticnet', None],
              'random_state': [40,50],
              'tol': [1,2,3]
            }
grid_log = GridSearchCV(clf1, parametrs, cv=10,scoring="f1_weighted")
grid_log.fit(X, y)

In [72]:
print(f'Лучшие параметры для логистической регрессии(rec1):{grid_log.best_params_}')
print(f'Лучший результат предсказания(rec1):{grid_log.best_score_}')

Лучшие параметры для логистической регрессии(rec1):{'penalty': 'l2', 'random_state': 40, 'tol': 1}
Лучший результат предсказания(rec1):0.986445458623731


In [74]:
clf1 = LogisticRegression(**grid_log.best_params_)
clf1 = clf1.fit(X_train, y_train)

In [75]:
cross_val_score(clf1, X, y)

array([0.99      , 0.98666667, 0.98833333, 0.97166667, 0.99      ])

# Gaussian Naive Bayes

In [79]:

# Создаем экземпляр модели GaussianNB
gnb = GaussianNB()

# Задаем набор гиперпараметров для перебора
param_grid = {'priors': [None, np.array([0.1, 0.9])],
              'var_smoothing': [1e-9, 1e-8, 1e-7]}

# Создаем экземпляр GridSearchCV
grid_search = GridSearchCV(gnb, param_grid, cv=5)

# Обучаем модель на обучающих данных
grid_search.fit(X_train, y_train)

# Выводим лучшие гиперпараметры и оценку качества на кросс-валидации
print("Лучшие гиперпараметры: {}".format(grid_search.best_params_))
print("Оценка качества на кросс-валидации: {:.2f}".format(grid_search.best_score_))

Лучшие гиперпараметры: {'priors': None, 'var_smoothing': 1e-09}
Оценка качества на кросс-валидации: 0.93


In [89]:
gnb = GaussianNB(**grid_search.best_params_)
gnb = gnb.fit(X_train, y_train)

# f1_score

In [90]:
print("Дерево решений:", f1_score(y_test, clf.predict(X_test),average='weighted'))
print("Логистическая регрессия:", f1_score(y_test, clf1.predict(X_test),average='weighted'))
print("Гауссов наивный байесовский метод ", f1_score(y_test, gnb.predict(X_test),average='weighted'))

Дерево решений: 0.9568252102645763
Логистическая регрессия: 0.9843262338020402
Гауссов наивный байесовский метод  0.9087411968296409


Вывод: Логистическая регрессия показала лучший результат